# Load Layer outputs

In [1]:
nn = 'NN'

import pickle
with open(nn+'_layer_outputs.dat','rb') as f:
    layer_outs,layer_outs_test= pickle.load(f)
f.close()

# Load recording and test data

In [2]:
import numpy as np
from keras.datasets import mnist
from scipy.misc import imresize

(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# Recording data
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

# downsample
factor = 0.25

Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(Xtest.shape[0]):
    Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(X_pr.shape[0]):
    X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
    
# VECTORIZE IMAGES
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

Using TensorFlow backend.


# Run XGBOOST

In [3]:
from copy import copy, deepcopy
from RE_PartialRecData import RE_PartialRecData
from RE_PartialRecData2 import RE_PartialRecData2
import os
import xgboost as xgb
import sklearn
from xgboost.sklearn import XGBClassifier
import pickle

params = {}
# use softmax multi-class classification 'multi:softmax'
# use linear regression 'reg:linear'
params['objective'] = 'reg:linear'
# scale weight of positive examples
params['eta'] = 0.5               # Makes the model more robust by shrinking the weights on each step (0.01-0.2)
params['max_depth'] = 6           # Used to control over-fitting as higher depth will allow model to learn relations 
                                  # very specific to a particular sample. (3-10)
params['silent'] = 1
params['nthread'] = 4
# params['num_class'] = 10
num_round = 5

# how many recordings?
nRecordings = 10
# how many neurons from the firs hidden layer?
subnetSize = [2**x for x in range(8)]
#subnetSize =[1, 64, 128]
nSubnetSize = len(subnetSize)
# which layers?
iLayer = 0
# how many samples per recording?
nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)*100
# how many iterations
nRecordings=1
nIterations = 5

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]
rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])
for iLayer in range(len(layer_outs)-1):
    for ss in range(nSubnetSize):
        #nLayerNeurons = [subnetSize[ss], 0, 0, 0, 10]
        nLayerNeurons = subnetSize[ss]
        print(subnetSize[ss])
        for it in range(nIterations):
            # copy data
            layer_outputs = deepcopy(layer_outs)
            # subsample
            X_subsample, Y_subsample = RE_PartialRecData2(layer_outputs[iLayer], layer_outputs[oLayer], 
                                                          nLayerNeurons, nRecordings,nSamples[ss])
    #        print('# nan values: ',np.count_nonzero(np.isnan(X_subsample)))
            # prepare data for xgboost
            for iN in range(nOutNeurons):
                #print('#neuron, #iteratin, subnetsize: ', iN,it,subnetSize[ss])
                xg_train  = xgb.DMatrix(X_subsample, label=Y_subsample[:, iN])
                xg_test   = xgb.DMatrix(layer_outs_test[iLayer], label=layer_outs_test[oLayer][:,iN])
                watchlist = [(xg_train, 'train'), (xg_test, 'test')]
                # train XGboost
                bst = xgb.train(params, xg_train, num_round, watchlist, verbose_eval=False)
                # get predictions
                pred = bst.predict(xg_test)
                rmses[it,iN,ss] = np.sqrt(np.mean(np.square([(pred[i] - layer_outs_test[oLayer][:,iN][i]) 
                                             for i in range(len(layer_outs_test[oLayer][:,1]))])))
            print ('predicting, RMSE=%f' %np.mean(rmses[it, :, ss]))


    fName = 'RMSES_Layer'+str(iLayer) + 'nRec' + str(nRecordings) + nn + '.dat'
    # save the rmse's
    with open(fName,'wb') as f:
        pickle.dump(rmses, f)
    f.close()


/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


1
predicting, RMSE=0.289300
predicting, RMSE=0.281589
predicting, RMSE=0.289766
predicting, RMSE=0.287154
predicting, RMSE=0.290196
2
predicting, RMSE=0.275260
predicting, RMSE=0.295555
predicting, RMSE=0.288545
predicting, RMSE=0.285438
predicting, RMSE=0.285824
4
predicting, RMSE=0.258583
predicting, RMSE=0.265166
predicting, RMSE=0.255939
predicting, RMSE=0.250892
predicting, RMSE=0.269266
8
predicting, RMSE=0.211106
predicting, RMSE=0.225384
predicting, RMSE=0.236220
predicting, RMSE=0.228899
predicting, RMSE=0.221539
16
predicting, RMSE=0.180820
predicting, RMSE=0.186924
predicting, RMSE=0.179276
predicting, RMSE=0.174495
predicting, RMSE=0.168423
32
predicting, RMSE=0.151299
predicting, RMSE=0.155106
predicting, RMSE=0.160294
predicting, RMSE=0.148241
predicting, RMSE=0.147483
64
predicting, RMSE=0.139328
predicting, RMSE=0.137070
predicting, RMSE=0.137934
predicting, RMSE=0.139624
predicting, RMSE=0.138194
128
predicting, RMSE=0.141222
predicting, RMSE=0.144081
predicting, RMSE=

In [4]:

# import matplotlib.pyplot as pl
# %matplotlib inline
# import numpy as np

# fig=pl.figure(figsize=(10,6))
# ax1 = fig.add_subplot(111)
# ax1.set_xlim([0, 128])
# ax2 = ax1.twiny()

# x = subnetSize;
# y = np.mean(np.median(rmses, axis=1), axis=0)
# error = np.std(np.mean(rmses, axis=1), axis=0)
# bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

# pl.plot(x, y, 'k-')
# # horiz_line_data = np.array([bl for i in xrange(len(x))])
# # pl.plot(x, horiz_line_data, 'k--') 
# pl.fill_between(x, y-error, y+error, alpha=0.2, facecolor='#808080')

# ax1.set_xlabel('# observed neurons on Layer 2 (out of 128)', fontsize=18)
# ax2.set_xlabel('Samples per recording',  fontsize=16)

# new_tick_locations = subnetSize
# ax1.set_xlim(ax1.get_xlim())
# ax1.set_xticks(new_tick_locations)
# ax1.set_xticklabels(new_tick_locations)

# new_tick_locations = [1,16,31,64,128]
# ax2.set_xlim(ax1.get_xlim())
# ax2.set_xticks(new_tick_locations)
# ax2.set_xticklabels(nSamples)


# # ax2.set_xticks(nSamples)
# # ax2.set_xticklabels(nSamples[range(0, 5, 20)])
# ax1.set_ylabel('RMSE', fontsize=18)
# ax1.set_ylim([0, .50])

# # pl.text(110,bl+0.005, 'baseline')


In [5]:
bl

NameError: name 'bl' is not defined